In [21]:
import pandas as pd
import os
from datetime import timedelta
import matplotlib.pyplot as plt

In [22]:
# column names : ReplicationNum, AirlineIdx, FlightDepTime, DepMarket, SSCPType, GrpSize, PaxArrTime, PaxSpeed, SSCPDelay, Visitors, LobbyDelay, DepFlightNumber, PaxType, PaxIDNum, PaxSSCPTime, PaxSPorPE

columns = ['ReplicationNum', 'AirlineIdx', 'FlightDepTime', 'DepMarket', 'SSCPType', 'GrpSize', 'PaxArrTime', 'PaxSpeed', 'SSCPDelay', 'Visitors', 'LobbyDelay', 'DepFlightNumber', 'PaxType', 'PaxIDNum']

In [23]:
def rolling_bin_max_sum(df, timeColumn, entityColumn, bin_interval=1, window=60, show_in_hhmm_format=False, plot=False):
    temp = df.copy()
    bins = pd.cut(temp[timeColumn], bins=range(0, 1441, bin_interval), right=False)
    
    grpSizeSum = temp.groupby(bins, observed=False)[entityColumn].sum()

    # Calculate rolling max of x-minute intervals
    rolling_sum = grpSizeSum.rolling(window).sum()
    rolling_max = rolling_sum.max()
    rolling_max_time = rolling_sum.idxmax()

    if plot:
        # plot
        rolling_sum_chart = rolling_sum.copy()
        rolling_sum_chart.index = rolling_sum.index.map(lambda x: x.left)
        rolling_sum_df = rolling_sum_chart.reset_index()    
        rolling_sum_df.columns = ['Time', 'Rolling Sum']
        rolling_sum_df.plot(x='Time', y='Rolling Sum', kind='line')
        plt.show()

    if show_in_hhmm_format:
        # convert to HH:MM format
        rolling_max_time_in_hhmm = str(timedelta(minutes=int(rolling_max_time.left)))[:-3]

        tempDF = pd.DataFrame({'Rolling Max': [int(rolling_max)], 'Time': [rolling_max_time_in_hhmm]})
        return tempDF
    else:

        rolling_max_time = rolling_max_time.left

        tempDF = pd.DataFrame({'Rolling Max': [int(rolling_max)], 'Time': [rolling_max_time]})
        return tempDF


In [24]:
def rolling_bin_max_sum_grouped(df, timeColumn, entityColumn, bin_interval=1, window=60, groupBy='', show_in_hhmm_format=False, plot=False):
    # Create bins for the given time column (0 to 1440 minutes, at bin_interval granularity)
    bins = pd.cut(df[timeColumn], bins=range(0, 1441, bin_interval), right=False)
    
    # Dictionary to store results for each PaxType
    results = {'PaxType': [], 'RollingMax': [], 'RollingMaxTime': []}
    
    # Iterate through each unique PaxType (groupBy category)
    for pax_type in df[groupBy].unique():
        temp_pax_type = df[df[groupBy] == pax_type]

        # Group by the time bins and sum the entity column within each bin
        grpSizeSum = temp_pax_type.groupby(bins, observed=False)[entityColumn].sum()

        # Calculate rolling sum and rolling max over the window
        rolling_sum = grpSizeSum.rolling(window=min(window, len(grpSizeSum)), min_periods=1).sum()  # Ensure rolling doesn't fail on small windows
        rolling_max = rolling_sum.max()
        rolling_max_time = rolling_sum.idxmax()

        # Check if the rolling sum is not empty
        if pd.notna(rolling_max_time):
            rolling_max_time_value = rolling_max_time.left  # Extract the left side of the interval
        else:
            rolling_max_time_value = None

        # Convert rolling max time to HH:MM format if required
        if show_in_hhmm_format and rolling_max_time_value is not None:
            rolling_max_time_value = str(timedelta(minutes=int(rolling_max_time_value)))[:-3]

        # Append results for this PaxType
        results['PaxType'].append(pax_type)
        results['RollingMax'].append(int(rolling_max) if pd.notna(rolling_max) else 0)
        results['RollingMaxTime'].append(rolling_max_time_value if rolling_max_time_value is not None else 'N/A')
        
        if plot:
            fig, ax = plt.subplots()

            for pax_type in df[groupBy].unique():
                temp_pax_type = df[df[groupBy] == pax_type]
                grpSizeSum = temp_pax_type.groupby(bins, observed=False)[entityColumn].sum()
                rolling_sum_chart = grpSizeSum.rolling(window).sum()
                rolling_sum_chart.index = rolling_sum_chart.index.map(lambda x: x.left)
                rolling_sum_df = rolling_sum_chart.reset_index()
                rolling_sum_df.columns = ['Time', 'Rolling Sum']

                # show max values
                rolling_sum_df['Rolling Sum Max'] = rolling_sum_df['Rolling Sum'].max()
                rolling_sum_df['Rolling Sum Max Time'] = rolling_sum_df[rolling_sum_df['Rolling Sum'] == rolling_sum_df['Rolling Sum Max']]['Time']

                ax.plot(rolling_sum_df['Time'], rolling_sum_df['Rolling Sum'], label=f'PaxType {pax_type}')
                
                # Annotate max values and their times
                for i in range(len(rolling_sum_df)):
                    if rolling_sum_df['Rolling Sum'][i] == rolling_sum_df['Rolling Sum Max'][i]:
                        ax.annotate(f'Max: {rolling_sum_df["Rolling Sum Max"][i]}', (rolling_sum_df['Time'][i], rolling_sum_df['Rolling Sum Max'][i]), textcoords='offset points', xytext=(30, -15), ha='center')
                        ax.annotate(f'Time: {rolling_sum_df["Rolling Sum Max Time"][i]}', (rolling_sum_df['Time'][i], rolling_sum_df['Rolling Sum Max'][i]), textcoords='offset points', xytext=(30, -25), ha='center')

            ax.set_xlabel('Time')
            ax.set_ylabel('Rolling Sum')
            ax.legend(title='PaxType')
            ax.set_title('Rolling Sum for Each PaxType')

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)   
    
    return results_df

In [25]:
def getPeaks(folder):
    
    all_data, all_data_grouped = {}, {}
    
    # read all csv files in the folder
    files = os.listdir(folder)
    if len(files) == 0:
        return None
    
    for file in files:
        if file.endswith('.csv'):
            df = pd.read_csv(folder + '/' + file, header=None, names=columns)
            
            # if SSCPType == 1 or 2, then PaxSPorPE = 1
            # if SSCPType == 3 or 4, then PaxSPorPE = 2
            df['PaxSPorPE'] = df['SSCPType'].apply(lambda x: 1 if x in [1, 2] else 2)

            df['PaxSSCPTime'] = df['PaxArrTime'] + df['LobbyDelay']

            # sort by PaxSSCPTime
            df = df.sort_values(by='PaxSSCPTime')

            result = rolling_bin_max_sum_grouped(df, 'PaxSSCPTime', 'GrpSize', bin_interval=1, window=60, groupBy='PaxSPorPE', show_in_hhmm_format=False, plot=False)

            result_no_group = rolling_bin_max_sum(df, 'PaxSSCPTime', 'GrpSize', bin_interval=1, window=60, show_in_hhmm_format=False)

            # add to dictionary with filename as key   
            all_data[file] = result_no_group
            all_data_grouped[file] = result
            
            
    # sort by filename
    all_data = dict(sorted(all_data.items()))
    all_data_grouped = dict(sorted(all_data_grouped.items()))
    
    newDF = pd.concat(all_data.values(), keys=all_data.keys())
    newDF_grouped = pd.concat(all_data_grouped.values(), keys=all_data_grouped.keys())
    
    # save to csv
    # newDF.to_excel('peaks.xlsx')
    
    return newDF, newDF_grouped


In [26]:
res, res_grped = getPeaks('C:\\Users\\vnirmal\\Desktop\\RIC\\RIC_9_26')

In [27]:
res_grped = res_grped.reset_index()
res_grped.drop(columns=['level_1', 'RollingMax'], inplace=True)
res_grped

,level_0,PaxType,RollingMaxTime
0,RIC2025Pax1.csv,1,341
1,RIC2025Pax1.csv,2,339
2,RIC2025Pax10.csv,1,335
3,RIC2025Pax10.csv,2,341
4,RIC2025Pax2.csv,2,338
5,RIC2025Pax2.csv,1,332
6,RIC2025Pax3.csv,1,338
7,RIC2025Pax3.csv,2,332
8,RIC2025Pax4.csv,1,333
9,RIC2025Pax4.csv,2,336


In [28]:
# if 'level_0' has RIC2025Pax1.csv, then set 1
# if 'level_0' has RIC2025Pax2.csv, then set 2 and so on till 10
res_grped['level_0'] = res_grped['level_0'].apply(lambda x: int(x.split('Pax')[1].split('.')[0]))

# if PaxType is 1 then 10 else 20
res_grped['PaxType'] = res_grped['PaxType'].apply(lambda x: 10 if x == 1 else 20)

# rename level_0 to RepNum
res_grped.rename(columns={'level_0': 'RepNum'}, inplace=True)

# rename RollingMaxTime to EndTime
res_grped.rename(columns={'RollingMaxTime': 'EndTime'}, inplace=True)

# rename PaxType to New_SSCPType
res_grped.rename(columns={'PaxType': 'New_SSCPType'}, inplace=True)


In [29]:

# save as csv
res_grped.to_csv('peaks.csv', index=False)

In [30]:
print(res)

                    Rolling Max  Time
RIC2025Pax1.csv  0         1412   340
RIC2025Pax10.csv 0         1450   335
RIC2025Pax2.csv  0         1410   335
RIC2025Pax3.csv  0         1424   338
RIC2025Pax4.csv  0         1397   333
RIC2025Pax5.csv  0         1440   334
RIC2025Pax6.csv  0         1415   331
RIC2025Pax7.csv  0         1412   335
RIC2025Pax8.csv  0         1464   333
RIC2025Pax9.csv  0         1401   333
